In [13]:
import os
from PIL import Image
from collections import defaultdict
import shutil
import xml.etree.cElementTree as ET
from tqdm import tqdm
import glob

#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))    

C:\Users\Ciprian-Florin Ifrim\AppData\Local\Temp\ipykernel_17164\1603078478.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [14]:
# function that holds the box coordinates and chooses them based on image resolution
def pascal_voc_gen(resolution):
    print(f'Generating PASCAL VOC XML files for resolution {resolution}...')
    boxes_dict = {
        (964,797): [(19,131,568,156), (19,23,879,66), (80,582,931,609), (3,208,79,577), (80,208,931,577)], # crypto.com 964x797
        (964,794): [(18,213,634,238), (15,22,762,68), (65,638,933,670), (6,263,66,639), (65,263,933,639)], # crypto.com 964x794
        (964,767): [(20,131,624,158), (20,16,827,68), (78,582,930,614), (1,209,78,580), (78,209,930,580)], # crypto.com 964x767
        (964,737): [(15,130,631,162), (20,23,859,74), (95,582,933,608), (2,211,95,579), (95,211,933,579)], # crypto.com 964x737
        (1429,909): [(38,1,323,50), (1,49,416,114), (475,870,1357,902), (1366,109,1429,870), (475,109,1366,870)], # coinmarketcap.com 1429x909x909
        (1622,909): [(29,2,308,46), (1,43,386,113), (1530,110,1622,778), (474,870,1530,909), (474,110,1530,778)], # coinmarketcap.com 2nd 1622x909
        (832,807): [(34,1,689,54), (1,94,791,164), (57,788,823,807), (1,419,57,788), (57,419,824,788)], # coindesk.com 832x807
        (832,806): [(32,1,703,52), (1,94,721,164), (70,788,824,807), (1,419,57,788), (70,419,824,788)], # coindesk.com 832x806
        (832,805): [(32,1,703,52), (1,94,721,164), (70,788,824,807), (1,419,57,788), (70,419,824,788)], # coindesk.com 832x805
        (832,802): [(34,1,699,51), (1,86,691,161), (97,784,824,802), (1,410,97,784), (97,410,824,784)], # coindesk.com 832x802
        (832,800): [(34,1,705,56), (1,90,696,157), (70,778,824,798), (1,412,40,778), (40,412,824,778)], # coindesk.com 832x800
        (832,798): [(34,1,705,56), (1,90,696,157), (70,778,823,798), (1,412,69,778), (69,412,824,779)], # coindesk.com 832x798
        (832,797): [(36,1,696,54), (1,87,753,156), (45,778,822,797), (1,412,44,778), (45,412,822,778)], # coindesk.com 832x797
        (783,635): [(50,15,494,99), (1,174,665,249), (1,610,725,635), (724,275,783,610), (1,275,725,610)], # binance.com volatility with warnings 783x635
        (783,663): [(51,28,499,129), (1,202,609,278), (1,638,720,663), (720,309,783,638), (1,309,720,638)], # binance.com volatility with warnings 783x663
        (783,711): [(59,6,474,90), (1,254,596,327), (1,688,712,715), (712,372,783,688), (1,372,712,688)], # binance.com with volatility warnings 783x711
        (783,731): [(62,2,619,89), (1,268,541,343), (1,705,724,731), (724,380,782,705), (1,380,724,705)], # binance.com with volatility warnings 783x731
        (783,791): [(53,75,499,212), (1,327,613,403), (1,766,729,791), (729,438,783,766), (1,438,729,766)], # binance.com with volatility warnings 783x791
        (783,603): [(54,8,499,143), (1,150,611,205), (1,578,724,603), (724,251,783,575), (1,251,724,578)], # binance.com no warnings 783x603
        (783,599): [(55,3,439,134), (1,143,247,199), (1,574,730,599), (730,253,783,574), (1,253,730,574)], # binance.com no warnings 783x599
        (783,555): [(59,1,439,107), (1,106,532,158), (1,529,720,555), (720,197,783,529), (1,197,720,529)], # binance.com no warnings 783x555
        (783,531): [(60,1,488,99), (1,89,610,144), (1,505,724,531), (724,182,783,505), (1,182,724,505)], # binance.com no warnings 783x531
        (783,527): [(62,1,499,99), (1,86,540,143), (1,501,724,527), (724,180,783,501), (1,180,724,501)], # binance.com no warnings 783x527
        (783,575): [(52,1,652,62), (1,115,537,187), (1,551,707,575), (707,217,783,551), (1,217,707,551)], # binance.com with listing warning 783x575
        (783,603): [(53,1,540,144), (1,147,615,202), (1,578,724,603), (724,247,783,578), (1,247,724,578)], # binance.com no warnings 783x603
        (783,599): [(51,1,540,144), (1,145,615,202), (1,574,724,599), (724,247,783,574), (1,247,724,574)], # binance.com no warnings 783x599
        (783,597): [(48,1,540,144), (1,145,615,202), (1,572,724,597), (724,247,783,572), (1,247,724,572)], # binance.com no warnings 783x597
        (783,675): [(50,1,761,136), (1,213,655,290), (1,650,723,675), (723,313,783,650), (1,313,723,650)], # binance.com 783x675
        (783,651): [(55,3,727,137), (1,190,553,259), (1,625,730,651), (729,293,783,627), (1,293,729,626)], # binance.com with listing warning 783x651
        (783,627): [(56,1,728,90), (1,165,710,238), (1,603,720,627), (720,268,783,603), (1,268,720,603)], # binance.com with listing warning 783x627
        (783,596): [(171,2,753,57), (1,137,522,209), (1,570,713,596), (713,237,783,570), (1,237,713,570)], # binance.com with listing warning 783x596
        (783,715): [(61,1,688,90), (1,250,727,327), (1,689,723,715), (723,355,783,689), (1,355,723,689)] # binance.com with listing warning 783x715
    }

    boxes = boxes_dict.get(resolution)

    if boxes is None:
        print(f"No boxes found for resolution {resolution}")
        return
    else:
        return boxes

# Function to create an XML file
def create_xml(image_path, dataset_save_path, boxes, labels, img_width, img_height, img_depth):
    print(f'Creating XML file for {image_path}...')
    annotation = ET.Element("annotation")
    ET.SubElement(annotation, "folder").text = os.path.dirname(image_path)
    ET.SubElement(annotation, "filename").text = os.path.basename(image_path)
    ET.SubElement(annotation, "path").text = image_path

    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(img_width)
    ET.SubElement(size, "height").text = str(img_height)
    ET.SubElement(size, "depth").text = str(img_depth)

    for i in range(len(boxes)):
        object = ET.SubElement(annotation, "object")
        ET.SubElement(object, "name").text = labels[i]
        ET.SubElement(object, "truncated").text = str(0)
        ET.SubElement(object, "difficult").text = str(0)

        bndbox = ET.SubElement(object, "bndbox")
        ET.SubElement(bndbox, "xmin").text = str(boxes[i][0])
        ET.SubElement(bndbox, "ymin").text = str(boxes[i][1])
        ET.SubElement(bndbox, "xmax").text = str(boxes[i][2])
        ET.SubElement(bndbox, "ymax").text = str(boxes[i][3])

    tree = ET.ElementTree(annotation)
    tree.write(dataset_save_path + os.path.splitext(os.path.basename(image_path))[0] + ".xml")
    print(f'XML file for {image_path} created successfully.')


image_folder = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Downsampled_NoAugmentation/JPEGImages'  
image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]             # Get a list of all your images
dataset_save_path = 'pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Downsampled_NoAugmentation/Annotations/'
labels = ["name", "value","x-axis", "y-axis", "plot"]                                           # Corresponding labels

# Create an XML file for each image
for image_path in tqdm(image_paths, desc="Creating XML files"):
    with Image.open(image_path) as img:  # Open the image file
        img_width, img_height = img.size
        img_depth = len(img.getbands())
        boxes = pascal_voc_gen((img_width, img_height))  # Assuming pascal_voc_gen is defined
        create_xml(image_path, dataset_save_path, boxes, labels, img_width, img_height, img_depth)

Creating XML files:   0%|                                                                              | 0/144888 [00:00<?, ?it/s]

Generating PASCAL VOC XML files for resolution (697, 512)...
No boxes found for resolution (697, 512)
Creating XML file for pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Downsampled_NoAugmentation/JPEGImages\image1.jpg...


TypeError: object of type 'NoneType' has no len()